In [46]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import fft , fftfreq
from scipy.io import wavfile

def get_wav_spectral_prop(filename) -> dict:
    fs, data = wavfile.read(filename)
    #get a 5 second sample from sound
    data = data[:5*fs]
    #normalize to [-1,1) range
    ndata = data.astype(np.float32)
    ndata = (ndata / np.max(np.abs(ndata)))
    ndata -= np.mean(ndata)
    #perform fft
    cfft = fft(ndata)
    d = int(len(cfft)/2)-1
    cfft = cfft[:d]
    rfft = abs(cfft)
    freqs = fftfreq(ndata.shape[0],1/fs)
    freqs = freqs[:d]
    maxf = freqs[np.where(rfft == np.amax(rfft))]
    result_d = {
        'max': maxf[0]
    }
    return result_d



def spectral_properties(y: np.ndarray, fs: int) -> dict:
    spec = np.abs(np.fft.rfft(y))
    freq = np.fft.rfftfreq(len(y), d=1 / fs)
    spec = np.abs(spec)
    amp = spec / spec.sum()
    mean = (freq * amp).sum()
    sd = np.sqrt(np.sum(amp * ((freq - mean) ** 2)))
    amp_cumsum = np.cumsum(amp)
    median = freq[len(amp_cumsum[amp_cumsum <= 0.5]) + 1]
    mode = freq[amp.argmax()]
    Q25 = freq[len(amp_cumsum[amp_cumsum <= 0.25]) + 1]
    Q75 = freq[len(amp_cumsum[amp_cumsum <= 0.75]) + 1]
    IQR = Q75 - Q25
    z = amp - amp.mean()
    w = amp.std()
    skew = ((z ** 3).sum() / (len(spec) - 1)) / w ** 3
    kurt = ((z ** 4).sum() / (len(spec) - 1)) / w ** 4

    result_d = {
        'mean': mean,
        'sd': sd,
        'median': median,
        'mode': mode,
        'Q25': Q25,
        'Q75': Q75,
        'IQR': IQR,
        'skew': skew,
        'kurt': kurt
    }
    return result_d


In [47]:
def hard_code_prediction(spectprop):
    if(spectprop['max'] > 173):
        return 'female'
    else:
        return 'male'

In [48]:
for i in range(0,12):
    print(f"analysing voice {i}...")
    sp = get_wav_spectral_prop(f"voices/v{i}.wav")
    pred = hard_code_prediction(sp)
    print(f"prediction: {pred}")

analysing voice 0...
prediction: female
analysing voice 1...
prediction: female
analysing voice 2...
prediction: male
analysing voice 3...
prediction: male
analysing voice 4...
prediction: female
analysing voice 5...
prediction: female
analysing voice 6...
prediction: male
analysing voice 7...
prediction: female
analysing voice 8...
prediction: male
analysing voice 9...
prediction: female
analysing voice 10...
prediction: female
analysing voice 11...
prediction: male
